In [2]:
import matplotlib.pyplot as plt
import matplotlib as mpl
from scipy.integrate import odeint,ode
import numpy as np
import pandas as pd
import math
# Uncomment the following line to get inline plots 
# %matplotlib inline

In [3]:
# define parameters for model
params = {
        'g_K'  :        30,       #min^-1,
        'g_L'  :        0.2,      #min^-1, 
        'V_K0' :       -380,      #mV,
        'V_L0' :       -156,      #mV,
        'S_th' :        40,
        'V_th' :       -150,
        'alpha_0' :     2,
        'beta' :        1.3,
        'm'    :        1,
        'F'    :        5.6,
        'sigma':        0.2,
    'delta_K'  :        1,
    'delta_L'  :        8,
    'gamma_s'  :        0.1,
    'gamma_e'  :        10,
    'gamma_t'  :        4,
    'alpha_s'  :        1,
    'alpha_t'  :        1,
    'D'        :        13.8*(10**(-6))   
}
# Define the value of e
e = math.exp(1)

In [4]:
def HodgkinHuxley(y0,t):
    V,n,S,E,T = y0
    
    V_k   = lambda E : params['V_K0'] + params['delta_K']*E
    V_l   = lambda E : params['V_L0'] + params['delta_L']*E    
    alpha = lambda S : (params['alpha_0']*(S**params['m'])) / (params['S_th']**params['m'] + S**params['m'])
    
    # all differential equations are wrt to time
    V_diff = lambda n,V : (-1)*params['g_K']*(n**4)*(V - V_k(E)) - params['g_L']*(V - V_l(E))
    n_diff = lambda S,n : alpha(S)*(1-n) - params['beta']*n
    # verify exp formula
    S_diff = lambda V,S : (params['alpha_s']*(params['V_th'] - V))/(e**((params['V_th'] - V)/params['sigma']) - 1 ) - params['gamma_s']*S                                         
    E_diff = lambda n,V,E : params['F']*params['g_K']*(n**4)*(V - V_k(E)) - params['gamma_e']*E
    T_diff = lambda V,T : params['alpha_t']*(params['V_L0'] - V) - params['gamma_t']*T
    
    return V_diff(n,V),n_diff(S,n),S_diff(V,S),E_diff(n,V,E),T_diff(V,T)

In [5]:
# initial values
V0 = -152.0
n0 = 0.10
S0 = 100.0
E0 = 100.0
T0 = 100.0

y0 = [V0,n0,S0,E0,T0]
# time interval spacing
h = .05
t_initial = 0
t_final = 100
t = np.arange(t_initial,t_final,h)

In [6]:
# Get the solutions, Note that each row gives values after one iteration
# Use X(:,i) i = 0,1,2 to access the membrane voltage, n, Stress value etc
X = odeint(HodgkinHuxley, y0, t )

In [7]:
# Put all the values in a Pandas Data frame and label the columns
sols = pd.DataFrame(X, columns=["V","n","S","E","T"])
sols.head()

,V,n,S,E,T
0,-152.000000,0.100000,100.000000,100.000000,100.000000
1,-145.837295,0.153979,99.583081,60.978719,81.535096
2,-142.368642,0.201029,99.389652,38.352367,66.205318
3,-140.740210,0.242064,99.321628,26.763686,53.542638
4,-140.447607,0.277866,99.300156,22.991942,43.134303


In [9]:
# We plot 4 variables V,T,S,E
plt.figure()
plt.subplot(4,1,1)
plt.title('Plots')

plt.subplot(4,1,1)
plt.plot(t, sols['V'])
plt.ylabel('V (mV)')
plt.legend()

plt.subplot(4,1,2)
plt.plot(t, sols['T'])
plt.ylabel('T in conc. units')
plt.legend()


plt.subplot(4,1,3)
plt.plot(t, sols['S'])
plt.ylabel('S in conc. units')
plt.legend()

plt.subplot(4,1,4)
plt.plot(t, sols['E'])
plt.ylabel('E in conc. units)')
plt.legend()
plt.show()

plt.close()